In [4]:
import livy, pprint, requests
host = livy.getHostUrl("http://34.205.23.216:9191")

In [5]:
session_url = livy.openSession(host)

...........................

In [6]:
upload_file_url = session_url + "/upload-file"

In [7]:
requests.post(upload_file_url, files={"file": open("gtex_rnaseq_prep_profiles.tsv","rb")})

<Response [200]>

In [109]:
code = '''
from pyspark import SparkContext
from pyspark import SparkFiles
from pyspark.ml.linalg import Vectors, Matrices
from pyspark.ml.stat import Correlation

path = SparkFiles.get("gtex_rnaseq_prep_profiles.tsv")

dataheaderindex = 2

with open(path) as inp:
    headers = inp.readline().strip().split("\t")
    # ids = []
    # features = []
    ncols = len(headers) - dataheaderindex
    nrows = 0
    data = []
    data2 = []
    for line in inp:
        row = line.strip().split('\t')
        if len(row) < dataheaderindex:
            continue
        # id = row[:dataheaderindex]
        feature = list(map(float, row[dataheaderindex:]))
        data.append(Vectors.dense(*feature))
        nrows += 1
        

rdd = sc.parallelize(data)

rddi = rdd.zipWithIndex()

def dot_self(v):
    return v.dot(v)
    
AA = rdd.map(dot_self).collect()
print(AA)

def tanimoto(veci):
    vec, i = veci
    data2 = [ABij / (AA[i] + AA[j] - ABij) for j, ABij in enumerate(map(lambda vec2 : vec.dot(vec2), data[:i]))]
    return data2
  
tanimotodf = rddi.map(tanimoto).collect()
%json tanimotodf
'''


In [110]:
r = livy.execStatement(host, session_url, code)

.

In [111]:
pprint.pprint(r.json())

{'code': '\n'
         'from pyspark import SparkContext\n'
         'from pyspark import SparkFiles\n'
         'from pyspark.ml.linalg import Vectors, Matrices\n'
         'from pyspark.ml.stat import Correlation\n'
         '\n'
         'path = SparkFiles.get("gtex_rnaseq_prep_profiles.tsv")\n'
         '\n'
         'dataheaderindex = 2\n'
         '\n'
         'with open(path) as inp:\n'
         '    headers = inp.readline().strip().split("\t")\n'
         '    # ids = []\n'
         '    # features = []\n'
         '    ncols = len(headers) - dataheaderindex\n'
         '    nrows = 0\n'
         '    data = []\n'
         '    data2 = []\n'
         '    for line in inp:\n'
         "        row = line.strip().split('\t')\n"
         '        if len(row) < dataheaderindex:\n'
         '            continue\n'
         '        # id = row[:dataheaderindex]\n'
         '        feature = list(map(float, row[dataheaderindex:]))\n'
         '        data.append(Vectors.dense(*fea

In [112]:
pprint.pprint(r.json()["output"]["data"]["application/json"])

[[],
 [0.09052962610042761],
 [0.5469027336657445, 0.0463593007011729],
 [0.21765022563652114, 0.19920991869585314, 0.14177260267525013],
 [0.058526821317668884,
  0.10894499485038078,
  0.04004578111452437,
  0.3813597620293863],
 [0.17152886376803891,
  0.02935936617124591,
  0.32940253215899457,
  0.07215944748111593,
  0.023264188539457883],
 [0.13174506772246786,
  0.013585239783545856,
  0.2764273791246808,
  0.03211720571867221,
  0.009562006372270074,
  0.07279029763518252],
 [0.7017407292708816,
  0.0967625559077445,
  0.6537697134549831,
  0.2684873542561289,
  0.07253269128488347,
  0.3111623304044326,
  0.14182648664537392],
 [0.6447813158200295,
  0.07694028418554551,
  0.48059849119313197,
  0.3052366563004832,
  0.07957998352843025,
  0.24111938099371677,
  0.09994087627075106,
  0.708298387545028]]


In [124]:
code = '''
from pyspark import SparkContext
from pyspark import SparkFiles
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

path = SparkFiles.get("gtex_rnaseq_prep_profiles.tsv")

dataheaderindex = 2

with open(path) as inp:
    headers = inp.readline().strip().split("\t")
    # ids = []
    # features = []
    data = []
    for line in inp:
        row = line.strip().split('\t')
        if len(row) < dataheaderindex:
            continue
        # id = row[:dataheaderindex]
        feature = map(float, row[dataheaderindex:])
        data.append(feature)
    data2 = list(map(lambda x : [Vectors.dense(*x)], zip(*data)))

df = spark.createDataFrame(data2, ["features"])

r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\\n" + str(r1))

r2 = Correlation.corr(df, "features", "spearman").head()
print("Spearman correlation matrix:\\n" + str(r2))
'''


In [125]:
r = livy.execStatement(host, session_url, code)

KeyError: 'location'

In [115]:
pprint.pprint(r.json())

{'code': '\n'
         'from pyspark import SparkContext\n'
         'from pyspark import SparkFiles\n'
         'from pyspark.ml.linalg import Vectors\n'
         'from pyspark.ml.stat import Correlation\n'
         '\n'
         'path = SparkFiles.get("gtex_rnaseq_prep_profiles.tsv")\n'
         '\n'
         'dataheaderindex = 2\n'
         '\n'
         'with open(path) as inp:\n'
         '    headers = inp.readline().strip().split("\t")\n'
         '    # ids = []\n'
         '    # features = []\n'
         '    data = []\n'
         '    for line in inp:\n'
         "        row = line.strip().split('\t')\n"
         '        if len(row) < dataheaderindex:\n'
         '            continue\n'
         '        # id = row[:dataheaderindex]\n'
         '        feature = map(float, row[dataheaderindex:])\n'
         '        data.append(feature)\n'
         '    data2 = list(map(lambda x : [Vectors.dense(*x)], zip(*data)))\n'
         '\n'
         'df = spark.createDataFrame(dat

In [116]:
pprint.pprint(r.json()["output"]["data"])

{'text/plain': 'Pearson correlation matrix:\n'
               'Row(pearson(features)=DenseMatrix(9, 9, [1.0, 0.2447, 0.2068, '
               '0.4914, 0.3069, -0.0747, 0.1278, 0.3957, ..., 0.0188, 0.2582, '
               '0.5914, 0.3462, 0.1979, 0.0447, 0.3077, 1.0], False))\n'
               'Spearman correlation matrix:\n'
               'Row(spearman(features)=DenseMatrix(9, 9, [1.0, 0.223, 0.3006, '
               '0.4801, 0.3286, 0.3539, 0.37, 0.5528, ..., 0.1393, 0.2525, '
               '0.4988, 0.3717, 0.4857, 0.1324, 0.3615, 1.0], False))'}


In [117]:
livy.closeSession(session_url)